### Import the needed libraries 

In [1]:
# Data manipulation and analysis
import pandas as pd

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Numerical operations
import numpy as np

# Date and time handling
from datetime import datetime

# Regular expression operations
import re

# Text and string manipulation
from unidecode import unidecode
import inflect

# Excel file handling
from pyxlsb import convert_date

# Progress bar for iterative tasks
from tqdm import tqdm
tqdm.pandas()


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

### Read The XLSB data file

In [2]:
df = pd.read_excel("../data/raw/data.xlsb")

In [3]:
df.head(3)

,Cod_cmd,Libellé produit,Vendeur,Univers,Nature,Date de commande,Montant cmd,Quantité,Prix transport,Délai transport annoncé
0,182210782,Table basse carrée detroit design industriel,Autre vendeur,Canapé Salon Séjour,Table basse,44216,244,4,6.67,10.0
1,182082437,Ours en peluche géant 150 cm brun,Autre vendeur,Enfant Bébé,Peluche,44213,28,1,9.92,10.0
2,182095765,Ours en peluche géant 100 cm blanc,Autre vendeur,Enfant Bébé,Peluche,44214,15,1,9.92,10.0


### Changing column names

In [4]:
column_name_mapping = {
    'Cod_cmd': 'command_code',
    'Libellé produit': 'product_description',
    'Vendeur': 'seller',
    'Univers': 'category',
    'Nature': 'nature',
    'Date de commande': 'order_date',
    'Montant cmd': 'order_amount',
    'Quantité': 'quantity',
    'Prix transport': 'transport_cost',
    'Délai transport annoncé': 'announced_transport_duration'
}

df.rename(columns=column_name_mapping, inplace=True)

### Correcting the Date column

In [5]:
def convert_xlsb_date_to_readable_format(xlsb_date):
    """
    Converts an Excel Binary Workbook (XLSB) date to a human-readable format.

    Args:
        xlsb_date (int): The Excel Binary Workbook date value.

    Returns:
        str: The date in a human-readable format.
    """
    return convert_date(xlsb_date)

df["order_date"] = df["order_date"].apply(lambda xlsb_date: convert_date(xlsb_date))

In [6]:
df.head(3)

,command_code,product_description,seller,category,nature,order_date,order_amount,quantity,transport_cost,announced_transport_duration
0,182210782,Table basse carrée detroit design industriel,Autre vendeur,Canapé Salon Séjour,Table basse,2021-01-20,244,4,6.67,10.0
1,182082437,Ours en peluche géant 150 cm brun,Autre vendeur,Enfant Bébé,Peluche,2021-01-17,28,1,9.92,10.0
2,182095765,Ours en peluche géant 100 cm blanc,Autre vendeur,Enfant Bébé,Peluche,2021-01-18,15,1,9.92,10.0


# Color & Dimension Extraction Algorithm ( Algo 2 )

### Dimension Extraction Function

In [7]:
def format_text(description):
    """
    Formats the dimensions in a product description to a standardized format.

    Args:
        description (str): The product description.

    Returns:
        str: Formatted description with correct dimension representation.
    """
    # Regular expression pattern to identify dimensions
    dimension_pattern = re.compile(r'(\d+)\s*([xX*])\s*(\d+)(?:\s*([xX*])\s*(\d+))*\s*(cm|MM|M|m|in|IN)?', re.IGNORECASE)

    def add_spaces(match):
        """Adds spaces between dimension parts."""
        parts = match.groups()
        formatted_dimensions = ' '.join(part if part else 'x' for part in parts if part is not None)
        return formatted_dimensions

    # Replace dimensions in the description with correct spaces
    formatted_description = dimension_pattern.sub(add_spaces, description)

    # Add a space before units (cm, MM, M, m, in, IN)
    formatted_description = re.sub(r'(\d)\s*(cm|MM|M|m|in|IN)', r'\1 \2', formatted_description, flags=re.IGNORECASE)

    return formatted_description


def extract_unit(input_string):
    """
    Extracts the unit (cm, mm, m, etc.) from the input string.

    Args:
        input_string (str): The input string containing dimensions.

    Returns:
        str: Extracted unit or None if no valid unit is found.
    """
    # Define a regular expression pattern to match the unit (cm, mm, m, etc.)
    pattern = r'\b(cm|mm|m|in|IN)\b'

    # Use re.search to find the pattern in the input string
    match = re.search(pattern, input_string, re.IGNORECASE)

    # Check if a match is found
    if match:
        # Extract and return the matched unit
        return match.group(1)
    else:
        # Return None if no valid unit is found
        return None


def extract_dimensions(description):
    """
    Extracts dimensions from product descriptions.

    Args:
        description (str): The product description.

    Returns:
        str: Extracted dimensions in a standardized format.
            Returns None if no valid dimensions are found.
    """
    def format_result(dim_string):
        """Formats the result"""
        # Extract numbers from the input string using regular expression
        numbers = re.findall(r'\b\d+\b', dim_string)

        # Join the numbers with '*' between them
        result = '*'.join(numbers)

        return result

    description = format_text(description)

    unit = extract_unit(description)

    if unit is not None and isinstance(description, str):
        # For any number of dimensions > 1
        regex_pattern = r'(\d+\s*[xX*]\s*\d+(?:\s*[xX*]\s*\d+)*)\s*(?:cm|MM|M|m|in|IN)?'
        dimension_match = re.search(regex_pattern, description)

        if dimension_match:
            dimensions = dimension_match.group(1)
            formatted_dimensions = format_result(dimensions)
            return formatted_dimensions

        # Handle the case of one dimension
        regex_single_dimension = r'(\d+)\s*(?:cm|MM|M|m|in|IN)?'
        single_dimension_match = re.search(regex_single_dimension, description)

        if single_dimension_match:
            # Check if re.search returns None before accessing group(1)
            dimension_group = re.search(r'\b(\d+)\s*(?:cm|mm|in|m)\b', description)
            if dimension_group:
                dimension = dimension_group.group(1)
                return dimension

    return None

In [8]:
# Example usage
example_with_single_dimension_cm = 'Product with 1 dimension 20 cm'
result_with_single_dimension_cm = extract_dimensions(example_with_single_dimension_cm)
print(example_with_single_dimension_cm)
print(f'>> {result_with_single_dimension_cm}\n')

example_with_two_dimensions_cm = 'Product with 2 dimensions 40x60 cm (without spaces)'
result_with_two_dimensions_cm = extract_dimensions(example_with_two_dimensions_cm)
print(example_with_two_dimensions_cm)
print(f'>> {result_with_two_dimensions_cm}\n')

example_with_three_dimensions_in = 'Product with 3 dimensions 20 x 100 x 30 IN (with spaces)'
result_with_three_dimensions_in = extract_dimensions(example_with_three_dimensions_in)
print(example_with_three_dimensions_in)
print(f'>> {result_with_three_dimensions_in}\n')

example_with_no_unit = 'Product with no unit 20 x 100 x 30 x 20'
result_with_no_unit = extract_dimensions(example_with_no_unit)
print(example_with_no_unit)
print(f'>> {result_with_no_unit}\n')

Product with 1 dimension 20 cm
>> 20

Product with 2 dimensions 40x60 cm (without spaces)
>> 40*60

Product with 3 dimensions 20 x 100 x 30 IN (with spaces)
>> 20*100*30

Product with no unit 20 x 100 x 30 x 20
>> None



### Apply the `Dimension Extraction Function` function on the dataframe

In [9]:
# Example usage on DataFrame df
df['extracted_dim'] = df['product_description'].progress_apply(lambda x: extract_dimensions(x) if isinstance(x, str) else None)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 525034/525034 [00:10<00:00, 51019.58it/s]


### Visualize a sample of the dataset

In [10]:
# Set the option to display the full content of each column without truncation
pd.set_option('display.max_colwidth', None)

# Generate a random sample of 9 rows from the DataFrame
random_sample = df.sample(n=9, random_state=1234567)

# Select specific columns ("product_description" and "extracted_dim") from the random sample
selected_columns = random_sample[["product_description", "extracted_dim"]]

# Display the selected columns with the full content of each row
selected_columns

,product_description,extracted_dim
276947,Buffet remy commode meuble de rangement avec 2 tiroirs et 2 portes en mélaminé décor ch ne sonoma,None
447105,Cadre à lattes fixe 160x200 cm,160*200
351696,Soleil d ocre oreiller a mémoire de forme 50x70 cm blanc,50*70
212607,Vasagle table de salle à manger pour 4 personnes table à d ner bureau 120 x 75 x 75 cm pour salle à manger cuisine cadre e,120*75*75
515491,Lave vaisselle 14 couverts,None
456136,Fauteuil relaxation électrique en tissu,None
427359,Canapé d angle convertible 4 places en tissu,None
293663,Cafetière filtre classique,None
223832,Reine 1w commode contemporaine chambre salon bureau 120x40x98 cm 8 tiroirs meuble de rangement scandinave dressing wen,120*40*98


### Color Extraction Function

In [11]:
def extract_colors(description):
    """
    Extracts colors from product descriptions.

    Args:
        description (str): The product description.

    Returns:
        str: Extracted colors in a formatted string.
              Returns an empty string if no valid colors are found.
    """
    # Define a set of extended color names including French color names
    valid_colors = set([
        'red', 'blue', 'green', 'yellow', 'black', 'white', 'orange', 'purple', 'pink',
        'brown', 'gray', 'navy', 'teal', 'maroon', 'olive', 'cyan', 'indigo', 'violet', 'gold',
        'rouge', 'bleu', 'vert', 'jaune', 'noir', 'blanc', 'orange', 'violet', 'rose',
        'marron', 'gris', 'marine', 'sarcelle', 'bordeaux', 'olive', 'cyan', 'indigo', 'violette', 'or'
        # Add more colors as needed
    ])

    # Generate the regular expression pattern dynamically
    color_pattern = r'\b(?:' + '|'.join(re.escape(color) for color in valid_colors) + r')\b'

    # Use re.findall to find all color matches in the description
    color_matches = re.findall(color_pattern, description, re.IGNORECASE)

    # Convert to lowercase and filter out colors that are not in the predefined set
    extracted_colors = [color.lower() for color in color_matches if color.lower() in valid_colors]

    # Return a formatted string
    return ', '.join(extracted_colors)

# Test the function
product_description = "This shirt is available in red, blue, and green colors. Il y a aussi une version en noir."
colors_found = extract_colors(product_description)

# Display the results
print("Product Description:", product_description)
print("Colors Found:", colors_found)

Product Description: This shirt is available in red, blue, and green colors. Il y a aussi une version en noir.
Colors Found: red, blue, green, noir


### Apply the `Color Extraction Function` function on the dataframe

In [12]:
# Example usage on DataFrame df
df['extracted_colors'] = df['product_description'].progress_apply(lambda x: extract_colors(x) if isinstance(x, str) else None)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 525034/525034 [00:25<00:00, 20705.39it/s]


In [13]:
# Set the option to display the full content of each column without truncation
pd.set_option('display.max_colwidth', None)

# Generate a random sample of 8 rows from the DataFrame
random_sample = df.sample(n=8, random_state=12354678)

# Select specific columns ("product_description" and "extracted_dim") from the random sample
selected_columns = random_sample[["product_description", "extracted_colors", "extracted_dim"]]

# Display the selected columns with the full content of each row
selected_columns

,product_description,extracted_colors,extracted_dim
285536,Verona colonne armoire de salle de bain 30x40x170 meuble salle de bain rangement salle de bain sonoma,,None
315339,Vasagle coiffeuse avec miroir et ampoules table de maquillage avec 5 tiroirs rangement cosmétique pieds en bois massif avec,,None
313498,Roam canapé convertible 3 places en tissu gris foncé,gris,None
278441,Lit structure lit adulte 140x190 classique en métal design noir,noir,None
403353,Tectake support tv mural pour ecran plat 32 à 60,,None
291858,Bibliothèque,,None
91965,Polaroid smart tv led hd 32 80cm wifi netflix screencast hdmi usb pvr 2 0 noir,noir,80
460615,Matelas ressorts 140x190 cm,,140*190


# Categorization Correction Algorithm ( Algo 1 )

### Viz The dataset

In [14]:
df_reduced = df[["product_description", "category", "nature"]][:10000]
df_reduced

,product_description,category,nature
0,Table basse carrée detroit design industriel,Canapé Salon Séjour,Table basse
1,Ours en peluche géant 150 cm brun,Enfant Bébé,Peluche
2,Ours en peluche géant 100 cm blanc,Enfant Bébé,Peluche
3,Lot de 4 chaises mia noires pour salle à manger,Canapé Salon Séjour,Chaise
4,Meuble tv falko bois blanc et gris,Canapé Salon Séjour,Meuble tv
...,...,...,...
9995,Rangement avec miroir elda,Cuisine Salle de bain,Elément Haut SDB
9996,Horloge silencieuse mécanique d51,Décoration Textile,Horloge
9997,Coffre à roulettes xxl fille,Enfant Bébé,Rangement enfant
9998,Tapis triangle ilan 120x170 cm ocre,Décoration Textile,Tapis de Salon et Ch


### Apply the `Correct Label Algorithm` function on the dataframe

In [15]:
def is_correctly_labeled(row):
    nature_value = f"{row['nature']}" if pd.notnull(row['nature']) else ''
    libelle_produit_value = f"{row['product_description']}" if pd.notnull(row['product_description']) else ''

    nature_normalized = re.sub(r'\W+', ' ', unidecode(nature_value).lower(), flags=re.UNICODE)
    libelle_normalized = re.sub(r'\W+', ' ', unidecode(libelle_produit_value).lower(), flags=re.UNICODE)

    p = inflect.engine()
    nature_words = {p.singular_noun(word) or word for word in nature_normalized.split()}
    libelle_words = {p.singular_noun(word) or word for word in libelle_normalized.split()}

    return any(word in libelle_words for word in nature_words)

# Use .loc to avoid the SettingWithCopyWarning
df_reduced['is_correctly_labeled'] = df_reduced.progress_apply(lambda row: is_correctly_labeled(row), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:36<00:00, 272.72it/s]


### Viz The detected incorrected Labbels 

In [16]:
df_reduced.query("is_correctly_labeled == False")

,product_description,category,nature,is_correctly_labeled
11,tagère à double position tania bois imitation h tre l 161 cm,Bureau Rangement,Bibliotheque,False
12,tagère à double position tania bois imitation h tre l 161 cm,Bureau Rangement,Bibliotheque,False
15,Console vintage leoni motifs graphiques,Bureau Rangement,Bureau,False
37,tagère à double position tania bois blanc l 161 cm,Bureau Rangement,Bibliotheque,False
38,tagère à double position tania bois blanc l 161 cm,Bureau Rangement,Bibliotheque,False
...,...,...,...,...
9993,Kit guitare électrique + amplificateur 15w + acccessoires blanc johnny brooks jb402,TV Son Multimédia,Sono lumière,False
9994,Fauteuil à bascule enfant moumoute,Canapé Salon Séjour,Chaise,False
9995,Rangement avec miroir elda,Cuisine Salle de bain,Elément Haut SDB,False
9997,Coffre à roulettes xxl fille,Enfant Bébé,Rangement enfant,False


### Function to calculate Jaccard similarity between two sets

In [17]:
def calculate_jaccard_similarity(set1, set2):
    """
    Calculate Jaccard similarity between two sets.

    Parameters:
    - set1, set2: Sets to calculate Jaccard similarity for.

    Returns:
    Jaccard similarity coefficient.
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Function to test calculate_jaccard_similarity with given test cases
def run_jaccard_similarity_tests():
    # Test case 1: Categories with common elements
    category_set1 = {'apple', 'orange', 'banana', 'kiwi'}
    category_set2 = {'banana', 'kiwi', 'grape', 'melon'}
    print(f'Test Case 1:')
    print(f'Set 1: {category_set1}')
    print(f'Set 2: {category_set2}')
    result1 = calculate_jaccard_similarity(category_set1, category_set2)
    print(f'Result: {result1}\n')

    # Test case 2: Categories with no common elements
    category_set3 = {'red', 'green', 'blue'}
    category_set4 = {'yellow', 'purple', 'pink'}
    print(f'Test Case 2:')
    print(f'Set 3: {category_set3}')
    print(f'Set 4: {category_set4}')
    result2 = calculate_jaccard_similarity(category_set3, category_set4)
    print(f'Result: {result2}\n')

# Run the test cases
run_jaccard_similarity_tests()

Test Case 1:
Set 1: {'kiwi', 'apple', 'banana', 'orange'}
Set 2: {'melon', 'kiwi', 'grape', 'banana'}
Result: 0.3333333333333333

Test Case 2:
Set 3: {'green', 'red', 'blue'}
Set 4: {'purple', 'yellow', 'pink'}
Result: 0.0



In [18]:
# Function to lemmatize a set of words
def lemmatize_words(word_set):
    lemmatizer = WordNetLemmatizer()
    return {lemmatizer.lemmatize(word) for word in word_set}

# Function to find the category with the best resemblance
def find_best_category(row, categories):
    """
    Find the category with the best resemblance to the 'product_description' column.

    Parameters:
    - row: DataFrame row.
    - categories: List of categories to compare against.

    Returns:
    Best matching category.
    """
    p = inflect.engine()

    description_value = row['product_description']
    if pd.isna(description_value):
        return None

    # Tokenize, lemmatize, and remove stopwords from the product_description column
    description_normalized = lemmatize_words({
        p.singular_noun(word) or word
        for word in re.findall(r'\b\w+\b', unidecode(str(description_value)).lower())
        if word.lower() not in stopwords.words('english')
    })

    best_category = None
    max_similarity = 0

    for category in categories:
        # Tokenize, lemmatize, and remove stopwords from each category
        category_words = lemmatize_words({
            p.singular_noun(word) or word
            for word in re.findall(r'\b\w+\b', unidecode(str(category)).lower())
            if word.lower() not in stopwords.words('english')
        })

        similarity = calculate_jaccard_similarity(description_normalized, category_words)

        if similarity > max_similarity:
            max_similarity = similarity
            best_category = category

    return best_category

In [19]:
# Apply the find_best_category function only to rows where is_correctly_labeled is False with progress bar
df_reduced['corrected_category'] = df_reduced[df_reduced['is_correctly_labeled'] == False].progress_apply(
    lambda row: find_best_category(row, df_reduced['nature'].unique()), axis=1
)

# Print the resulting DataFrame
df_reduced

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1248/1248 [06:14<00:00,  3.34it/s]


,product_description,category,nature,is_correctly_labeled,corrected_category
0,Table basse carrée detroit design industriel,Canapé Salon Séjour,Table basse,True,NaN
1,Ours en peluche géant 150 cm brun,Enfant Bébé,Peluche,True,NaN
2,Ours en peluche géant 100 cm blanc,Enfant Bébé,Peluche,True,NaN
3,Lot de 4 chaises mia noires pour salle à manger,Canapé Salon Séjour,Chaise,True,NaN
4,Meuble tv falko bois blanc et gris,Canapé Salon Séjour,Meuble tv,True,NaN
...,...,...,...,...,...
9995,Rangement avec miroir elda,Cuisine Salle de bain,Elément Haut SDB,False,Miroir
9996,Horloge silencieuse mécanique d51,Décoration Textile,Horloge,True,NaN
9997,Coffre à roulettes xxl fille,Enfant Bébé,Rangement enfant,False,Coffre
9998,Tapis triangle ilan 120x170 cm ocre,Décoration Textile,Tapis de Salon et Ch,True,NaN


In [20]:
df_reduced[df_reduced["is_correctly_labeled"]==False].head(30)

,product_description,category,nature,is_correctly_labeled,corrected_category
11,tagère à double position tania bois imitation h tre l 161 cm,Bureau Rangement,Bibliotheque,False,Jeu d'imitation
12,tagère à double position tania bois imitation h tre l 161 cm,Bureau Rangement,Bibliotheque,False,Jeu d'imitation
15,Console vintage leoni motifs graphiques,Bureau Rangement,Bureau,False,Console
37,tagère à double position tania bois blanc l 161 cm,Bureau Rangement,Bibliotheque,False,None
38,tagère à double position tania bois blanc l 161 cm,Bureau Rangement,Bibliotheque,False,None
41,Lot de 3 étagères murales grises,Bureau Rangement,Bibliotheque,False,Etagère
61,tagère échelle scandinave klara bois blanc et fa on h tre,Bureau Rangement,Bibliotheque,False,Toile et laminage
64,Ecouteur sans fil bluetooth pour smartphone et ipad blanc,TV Son Multimédia,Microphone,False,Casque sans fil
95,Meuble de rangement en escalier liam 4 niveaux bois blanc + porte tiroirs blancs fond gris,Bureau Rangement,Bibliotheque,False,Boite de rangement
114,Etagère échelle 4 niveaux detroit design industriel,Bureau Rangement,Bibliotheque,False,Etagère


In [21]:
incorrect_labels_count = df_reduced['is_correctly_labeled'].sum()
corrected_labels_count = len(df_reduced) - incorrect_labels_count

print(f"Number of Incorrect Labels: {incorrect_labels_count}")
print(f"Number of Corrected Labels: {corrected_labels_count}")
percentage_corrected = (corrected_labels_count / len(df_reduced)) * 100
print(f"Percentage of Labels Corrected: {percentage_corrected:.2f}%")

Number of Incorrect Labels: 8752
Number of Corrected Labels: 1248
Percentage of Labels Corrected: 12.48%
